# Using `bw2landbalancer`

Notebook showing typical usage of `bw2landbalancer`

## Generating the samples

`bw2landbalancer` works with Brightway2. You only need set as current a project in which the database for which you want to balance land transformation exchanges is imported.

In [1]:
import bw2data as bd
import bw2calc as bc
import numpy as np
import plotly.graph_objects as go
from scipy.stats import lognorm
from gsa_framework.utils import read_pickle

from bw2landbalancer import ActivityLandBalancer


bd.projects.set_current('GSA for archetypes') # Project with ecoinvent 3.6 cut-off by classification already imported
ei_name = "ecoinvent 3.8 cutoff"
ei = bd.Database(ei_name)
bio = bd.Database("biosphere3")
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")

co = bd.Database('swiss consumption 1.0')
fu = [act for act in co if "average consumption" in act['name']][0]
lca = bc.LCA({fu:1}, method=method)
lca.lci()
lca.lcia()
lca.score

1108.8133423074094

The only Class you need is the `DatabaseLandBalancer`:

In [2]:
from bw2landbalancer import DatabaseLandBalancer

Instantiating the DatabaseLandBalancer will automatically identify land transformation biosphere activities (elementary flows). 

In [3]:
dlb = DatabaseLandBalancer(
    database_name=ei_name,  # name the LCI db in the brightway2 project
)

Validating data
Getting information on land transformation exchanges


Generating presamples for the whole database is a lengthy process. Thankfully, it only ever needs to be done once per database:

In [4]:
# %%time
# dlb.add_samples_for_all_acts(iterations=2000)

The samples and associated indices are stored as attributes: 

In [5]:
# from gsa_framework.utils import write_pickle
# write_pickle(dlb.matrix_samples, "land_samples_2000.pickle")

In [6]:
# write_pickle(dlb.matrix_indices, "land_indices_2000.pickle")

In [7]:
matrix_samples = read_pickle("land_samples_2000.pickle")
matrix_indices = read_pickle("land_indices_2000.pickle")

FileNotFoundError: [Errno 2] No such file or directory: 'land_samples_2000.pickle'

In [ ]:
# dlb.matrix_indices[0:10] # First ten indices

In [ ]:
indices = lca.biosphere_mm.groups[1].package.get_resource("ecoinvent_3.8_cutoff_biosphere_matrix.indices")[0]
data = lca.biosphere_mm.groups[1].package.get_resource("ecoinvent_3.8_cutoff_biosphere_matrix.data")[0]
uncertainties = lca.biosphere_mm.groups[1].package.get_resource("ecoinvent_3.8_cutoff_biosphere_matrix.distributions")[0]

In [ ]:
id_row = bd.get_id(matrix_indices[0][0])
id_col = bd.get_id(matrix_indices[0][1])
row = lca.dicts.biosphere[id_row]
col = lca.dicts.activity[id_col]
row, col

In [ ]:
from pypardiso import spsolve
B = lca.biosphere_matrix
B[548, 10650] *= 155
imp = lca.characterization_matrix*B* spsolve(lca.technosphere_matrix, lca.demand_array)
imp.sum()

In [ ]:
m = bd.Method(method)

In [ ]:
for exc in m.load():
    print(bd.get_activity(exc[0])['name'])

In [ ]:
aa = np.array((id_row, id_col), dtype=indices.dtype)
np.where(indices==aa)

In [ ]:
data[103454]

In [ ]:
dist_arr
np.where(dist_arr>100000)[0].shape

In [ ]:
num_bins=400
dist = []
len_ = len(matrix_indices)
aa = 1532
for i, ind in enumerate(matrix_indices[aa:aa+1]):
    if i%100==0:
        print(f"{i}/{len_}")
    # Landbalancing samples
    Y = matrix_samples[i,:]
    bin_min = min(Y)
    bin_max = max(Y)
    bins_ = np.linspace(bin_min, bin_max, num_bins, endpoint=True)
    Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
    # Given distribution
    row = bd.get_id(ind[0])
    col = bd.get_id(ind[1])
    row_col = np.array((row,col), dtype=indices.dtype)
    where = np.where(indices == row_col)[0][0]
    unct = uncertainties[where:where+1]
    if unct['uncertainty_type']==2:
        num_bins = 60
        loc = unct['loc']
        scale = unct['scale']  
        midbins = (bins_[1:]+bins_[:-1])/2
        Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
    else:
        print(i, unct['uncertainty_type'])
    if len(Y_samples)==0:
        print(f"{i} here")
    dist.append(sum((Y_samples-Y_distr)**2)/len(Y_samples))

In [8]:
ind

NameError: name 'ind' is not defined

In [9]:
bd.get_activity(('biosphere3', '2f1e926a-ec96-432b-b2a6-bd5e3de2ff87'))

'Transformation, to arable land, unspecified use' (square meter, None, ('natural resource', 'land'))

In [10]:
t = bd.get_activity(('ecoinvent 3.8 cutoff', 'f7221f847f32280677ba1bd9a1c90dda'))

In [11]:
list(t.exchanges())

[Exchange: 1.82550304055e-05 kilogram 'market for organophosphorus-compound, unspecified' (kilogram, GLO, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 0.00156219174204059 kilogram 'market for potassium sulfate' (kilogram, RoW, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 1.20483200676e-06 kilogram 'market for pyrethroid-compound' (kilogram, GLO, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 1.0 kilogram 'onion production' (kilogram, NZ, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 0.00103666666666872 kilogram 'market for packaging, for pesticides' (kilogram, GLO, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 4.62947571082e-05 kilogram 'market for dinitroaniline-compound' (kilogram, GLO, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 1.37277828649e-05 kilogram 'market for cyclic N-compound' (kilogram, GLO, None) to 'onion production' (kilogram, NZ, None)>,
 Exchange: 25.1111111111 unit 'mar

In [12]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = midbins,
        y = Y_samples,
        line_color = 'blue',
    )
)
fig.add_trace(
    go.Scatter(
        x = midbins,
        y = Y_distr,
        line_color = 'red',
    )
)
fig.show()

NameError: name 'midbins' is not defined

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = midbins,
        y = Y_samples,
        line_color = 'blue',
    )
)

In [ ]:
# ind = 149
# row = bd.get_id(matrix_indices[ind][0])
# col = bd.get_id(matrix_indices[ind][1])
# row_col = np.array((row,col), dtype=indices.dtype)
# where = np.where(indices == row_col)[0][0]
# unct = uncertainties[where:where+1]
# assert unct['uncertainty_type']==2

# Y = matrix_samples[ind,:]
# bin_min = min(Y)
# bin_max = max(Y)
# num_bins = 60
# loc = unct['loc']
# scale = unct['scale']
# x = np.linspace(bin_min, bin_max, 100) 
     
# # Varying positional arguments 
# y_distr = lognorm.pdf(x, s=scale, scale=np.exp(loc))

# fig = go.Figure()
# bins_ = np.linspace(bin_min, bin_max, num_bins, endpoint=True)
# freq, bins = np.histogram(Y, bins=bins_, density=True)

# fig.add_trace(
#     go.Scatter(
#         x=bins,
#         y=freq,
#         opacity=0.65,
#         line=dict(color="blue", width=1, shape="hvh"),
#         fill="tozeroy",
#     ),
# )

# fig.add_trace(
#     go.Scatter(
#         x = x,
#         y = y_distr,
#         line_color = 'red',
#     )
# )
# fig.show()

In [ ]:
a = [act for act in ei if 'market for electricity, low' in act['name']]
a

In [ ]:
list(a[0].exchanges())

In [ ]:
"3b15ec72f45f4700fec86f75084fe558", "666aa058fa7418ec809f91cb610720ad"

In [ ]:
a[0].as_dict()['reference product'], a[-4].as_dict()['reference product']

In [ ]:
# bio_inds = lca.packages[1].get_resource("ecoinvent_3.8_cutoff_biosphere_matrix.indices")[0]
# tfrom = [act.id for act in bio if "Transformation, from" in act['name']]
# tto = [act.id for act in bio if "Transformation, to" in act['name']]
# land_acts = tfrom + tto

In [ ]:
# ei_acts = np.array([], dtype=int)
# for a in land_acts:
#     mask = bio_inds['row'] == a
#     ei_acts = np.hstack([ei_acts, bio_inds[mask]['col']])
# ei_acts.sort()
# ei_acts = np.unique(ei_acts)
# ei_acts.shape

In [ ]:
# for ei_act_id in ei_acts:
#     ei_act = bd.get_activity(ei_act_id)
#     act_balancer = ActivityLandBalancer(ei_act.key, dlb)
#     act_balancer._identify_strategy()
#     if act_balancer.strategy == 'static':
#         break

In [ ]:
# type(4435)

In [ ]:
# bd.get_activity(4435)

In [ ]:
# ei_act_id, type(ei_act_id)

In [ ]:
# bd.get_activity(int(ei_act_id))

In [ ]:
# bd.get_activity(ei_act_id)

In [ ]:
# for x in dlb.matrix_indices:
#     a = bd.get_activity(x[1])
#     from bw2landbalancer import ActivityLandBalancer
#     aa = ActivityLandBalancer(a.key, dlb)
#     if aa.activity_params:
#         break

In [ ]:
# list(aa.act.exchanges())

These can directly be used to generate [`presamples`](https://presamples.readthedocs.io/):

In [ ]:
presamples_id, presamples_fp = dlb.create_presamples(
    name=None,  # Could have specified a string as name, not passing anything will use automatically generated random name
    dirpath=None,  # Could have specified a directory path to save presamples somewhere specific 
    id_=None,  # Could have specified a string as id, not passing anything will use automatically generated random id
    seed='sequential',  # or None, or int.
    )

## Using the samples

The samples are formatted for use in brighway2 via the presamples package. 

The following function calculates:  
  - Deterministic results, using `bc.LCA`  
  - Stochastic results, using `bc.MonteCarloLCA`  
  - Stochastic results using presamples, using `bc.MonteCarloLCA` and passing `presamples=[presamples_fp]`  
  
The ratio of stochastic results to deterministic results are then plotted for Monte Carlo results with and without presamples.  
Ratios for Monte Carlo with presamples are on the order of 1.  
Ratios for Monte Carlo without presamples can be multiple orders of magnitude, and can be negative or positive.    

In [ ]:
def check_presamples_act(act_key, ps_fp, lcia_method, iterations=1000):
    """Plot histrograms of Monte Carlo samples/det result for case w/ and w/o presamples"""
    lca = bc.LCA({act_key:1}, method=m)
    lca.lci()
    lca.lcia()
    
    mc_arr_wo = np.empty(shape=iterations)
    mc = bc.MonteCarloLCA({act_key:1}, method=m)
    for i in range(iterations):
        mc_arr_wo[i] = next(mc)/lca.score
    
    mc_arr_w = np.empty(shape=iterations)
    mc_w = bc.MonteCarloLCA({act_key:1}, method=m, presamples=[ps_fp])
    for i in range(iterations):
        mc_arr_w[i] = next(mc_w)/lca.score
    
    plt.hist(mc_arr_wo, histtype="step", color='orange', label="without presamples")
    plt.hist(mc_arr_w, histtype="step", color='green', label="with presamples")
    plt.legend()

Let's run this on a couple of random ecoinvent products with the ImpactWorld+ Land transformation, biodiversity LCIA method:

In [ ]:
m=('IMPACTWorld+ (Default_Recommended_Midpoint 1.23)', 'Midpoint', 'Land transformation, biodiversity')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
act = [act for act in bd.Database(ei_name) if act['name']=='polyester-complexed starch biopolymer production'][0]
print("Working on activity known to have non-negligeable land transformation impacts: ", act)
check_presamples_act(act.key, presamples_fp, m)

In [ ]:
act = bd.Database(ei_name).random()
print("Randomly working on ", act)
check_presamples_act(act.key, presamples_fp, m)

In [ ]:
act = bd.Database(ei_name).random()
print("Randomly working on ", act)
check_presamples_act(act.key, presamples_fp, m)

In [ ]:
act = bd.Database(ei_name).random()
print("Randomly working on ", act)
check_presamples_act(act.key, presamples_fp, m)

In [ ]:
act = bd.Database(ei_name).random()
print("Randomly working on ", act)
check_presamples_act(act.key, presamples_fp, m)